In [1]:
import requests
import pandas as pd
import geopandas as gpd

In [7]:
link_prefix = "https://raw.githubusercontent.com/giswqs/data/main/housing/"


In [8]:
data_links = {

    "monthly_current": {
        "state": link_prefix + "Core/RDC_Inventory_Core_Metrics_State.csv",
        "county": link_prefix + "Core/RDC_Inventory_Core_Metrics_County.csv",
    },
}

In [2]:
prefix = (
    "https://raw.githubusercontent.com/giswqs/streamlit-geospatial/master/data/"
)
links = {
    "national": prefix + "us_nation.geojson",
    "state": prefix + "us_states.geojson",
    "county": prefix + "us_counties.geojson",
    "metro": prefix + "us_metro_areas.geojson",
    "zip": "https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_zcta510_500k.zip",
}

# if category.lower() == "zip":
#     r = requests.get(links[category])
#     out_zip = os.path.join(DOWNLOADS_PATH, "cb_2018_us_zcta510_500k.zip")
#     with open(out_zip, "wb") as code:
#         code.write(r.content)
#     zip_ref = zipfile.ZipFile(out_zip, "r")
#     zip_ref.extractall(DOWNLOADS_PATH)
#     gdf = gpd.read_file(out_zip.replace("zip", "shp"))
# else:
#     gdf = gpd.read_file(links[category])
# return gdf

In [4]:
gdf = gpd.read_file(links["county"])

In [9]:
def get_inventory_data(url):
    df = pd.read_csv(url)
    url = url.lower()
    if "county" in url:
        df["county_fips"] = df["county_fips"].map(str)
        df["county_fips"] = df["county_fips"].str.zfill(5)
    elif "state" in url:
        df["STUSPS"] = df["state_id"].str.upper()
    elif "metro" in url:
        df["cbsa_code"] = df["cbsa_code"].map(str)
    elif "zip" in url:
        df["postal_code"] = df["postal_code"].map(str)
        df["postal_code"] = df["postal_code"].str.zfill(5)

    if "listing_weekly_core_aggregate_by_country" in url:
        columns = get_data_columns(df, "national", "weekly")
        for column in columns:
            if column != "median_days_on_market_by_day_yy":
                df[column] = df[column].str.rstrip("%").astype(float) / 100
    if "listing_weekly_core_aggregate_by_metro" in url:
        columns = get_data_columns(df, "metro", "weekly")
        for column in columns:
            if column != "median_days_on_market_by_day_yy":
                df[column] = df[column].str.rstrip("%").astype(float) / 100
        df["cbsa_code"] = df["cbsa_code"].str[:5]
    return df

In [14]:
get_inventory_data(data_links["monthly_current"]["county"])

,month_date_yyyymm,county_fips,county_name,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,median_square_feet_yy,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy
0,202109,37001,"alamance, nc",294995.0,0.0077,-0.0084,288,0.0909,-0.1933,21.5,...,-0.0814,3.798707e+05,-0.0188,0.0948,666,0.0622,0.0168,1.312500,-0.0625,0.4778
1,202109,42079,"luzerne, pa",184500.0,-0.0261,0.0256,432,-0.0379,-0.1955,35.5,...,-0.0245,2.597044e+05,0.0170,0.0302,530,0.0000,-0.1396,0.226852,0.0465,0.0797
2,202109,42091,"montgomery, pa",407450.0,-0.0074,-0.1283,1149,-0.0312,-0.1058,46.0,...,-0.1823,6.009249e+05,0.0014,-0.1484,3096,-0.0621,0.7286,1.694517,-0.0888,1.3007
3,202109,27111,"otter tail, mn",277700.0,0.0238,0.0289,236,0.0085,-0.2081,52.5,...,0.0047,3.802250e+05,-0.0203,0.0619,398,-0.0100,-0.0124,0.686441,-0.0315,0.3341
4,202109,18127,"porter, in",348495.0,0.0253,0.0029,242,0.0852,0.1101,30.5,...,-0.1089,4.709305e+05,0.0465,0.0682,384,0.0435,-0.0965,0.586777,-0.0634,-0.3628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,202109,25021,"norfolk, ma",699900.0,0.0000,0.0013,831,0.0695,-0.2653,27.5,...,-0.0378,1.045647e+06,-0.0148,-0.0394,1028,0.0301,-0.2704,0.237064,-0.0474,-0.0087
996,202109,06025,"imperial, ca",327500.0,0.0364,0.1618,81,-0.0581,-0.0357,44.5,...,0.0651,3.655621e+05,0.0081,0.2604,248,0.0000,0.4850,2.061728,0.1780,1.0736
997,202109,48469,"victoria, tx",280000.0,-0.0087,0.0138,172,0.0683,-0.1925,45.0,...,-0.1271,3.410071e+05,0.0139,0.0274,289,0.0396,-0.1189,0.680233,-0.0465,0.1403
998,202109,26117,"montcalm, mi",219500.0,-0.0557,0.2201,115,0.0849,0.1386,39.5,...,-0.0101,2.545913e+05,-0.0662,0.1398,257,0.0000,-0.0982,1.234783,-0.1897,-0.5870


In [24]:
df = get_inventory_data(data_links["monthly_current"]["state"])

In [25]:
df

,month_date_yyyymm,state,state_id,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,STUSPS
0,202109,colorado,co,575000.0,-0.0070,0.1275,12382,-0.0075,-0.2945,37.0,...,1.157431e+06,-0.0298,0.1000,25980,-0.0163,-0.2629,1.098207,-0.0187,0.0900,CO
1,202109,north dakota,nd,254950.0,-0.0190,0.0475,2020,-0.0015,-0.2681,59.5,...,3.011814e+05,-0.0154,0.0610,3158,-0.0150,-0.2197,0.563366,-0.0214,0.0971,ND
2,202109,utah,ut,572450.0,-0.0325,0.1204,5607,0.0772,-0.0870,27.0,...,1.061615e+06,-0.0434,0.0897,15062,0.0311,-0.0872,1.686285,-0.1201,-0.0007,UT
3,202109,vermont,vt,359450.0,0.0270,0.1062,1569,-0.0694,-0.4034,58.5,...,5.600091e+05,0.0269,0.1331,2723,-0.0347,-0.3760,0.735500,0.0623,0.0762,VT
4,202109,iowa,ia,219500.0,-0.0277,0.0097,7826,0.0093,-0.2530,52.0,...,2.759342e+05,-0.0083,0.0745,15625,-0.0289,-0.1584,0.996550,-0.0785,0.2246,IA
5,202109,arkansas,ar,229950.0,0.0020,0.0950,6255,0.0204,-0.2191,45.5,...,3.392705e+05,0.0097,0.1517,12448,0.0067,-0.0125,0.990088,-0.0270,0.4164,AR
6,202109,indiana,in,232500.0,-0.0085,0.0222,9137,0.0453,-0.1081,36.5,...,3.148671e+05,-0.0022,0.0487,17070,0.0137,-0.0895,0.868228,-0.0583,0.0382,IN
7,202109,louisiana,la,269900.0,-0.0185,0.0645,8515,-0.0293,-0.2878,64.5,...,3.809970e+05,-0.0017,0.1097,17692,-0.0321,-0.1066,1.077745,-0.0060,0.4213,LA
8,202109,new york,ny,574000.0,0.0088,0.0132,59417,0.0007,-0.1397,70.0,...,1.347206e+06,0.0193,0.1469,96447,0.0016,-0.1506,0.623222,0.0014,-0.0208,NY
9,202109,california,ca,726500.0,-0.0082,0.0028,46633,-0.0010,-0.1170,39.0,...,1.535083e+06,-0.0054,0.0254,99355,-0.0041,-0.1158,1.130573,-0.0067,0.0027,CA
